In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup
import psycopg2

In [2]:
df = pd.read_csv('./datasets/companylisting.csv')
tickers = df['ticker'].values

In [3]:
shares_outstanding = []
for t in tickers:
    try:
        r = requests.get('https://www.marketwatch.com/investing/stock/{}'.format(t.replace('-', '.')))
        soup = bs4.BeautifulSoup(r.text, "html.parser")
        if soup.findAll('span', {'class':'kv__value kv__primary'})[4].text.strip().lower() == 'n/a':
            shares_outstanding.append(soup.findAll('span', {'class':'kv__value kv__primary'})[5].text.strip())
        else:
            shares_outstanding.append(soup.findAll('span', {'class':'kv__value kv__primary'})[4].text.strip())
    except:
        print('Error Retrieving Data for {}'.format(t))
        shares_outstanding.append(np.nan)

Error Retrieving Data for HRZN


In [51]:
column_names = ['ticker', 'shares_outstanding']
df = []
for i in range(len(tickers)):
    df.append([tickers[i], shares_outstanding[i]])

In [52]:
df = pd.DataFrame(df, columns=column_names)

In [53]:
for i,v in enumerate(df['shares_outstanding']):
    try:
        print(float(v), i)
    except:
        continue

nan 329
37.67 477
29.29 711


In [54]:
for i,v in enumerate(df['shares_outstanding']):
    try:
        if '$' in v or 'K' in v:
            print(v, i)
    except:
        continue

723K 104
$738.18K 143
$266.34K 429


In [55]:
df.loc[143]['shares_outstanding'] = '2.45B'
df.loc[329]['shares_outstanding'] = '11.52M'
df.loc[429]['shares_outstanding'] = '46.79M'
df.loc[477]['shares_outstanding'] = '320.71M'
df.loc[711]['shares_outstanding'] = '269.9M'

In [56]:
for i in range(len(df)):
    if type(df.loc[i]['shares_outstanding']) == str:
        if 'M' in df.loc[i]['shares_outstanding']:
            df.loc[i]['shares_outstanding'] = float(df.loc[i]['shares_outstanding'].replace('M', '')) * 1000000
        elif 'B' in df.loc[i]['shares_outstanding']:
            df.loc[i]['shares_outstanding'] = float(df.loc[i]['shares_outstanding'].replace('B', '')) * 1000000000
        else:
            df.loc[i]['shares_outstanding'] = float(df.loc[i]['shares_outstanding'].replace('K', '')) * 1000
    else:
        continue

In [58]:
df.to_csv('./datasets/shares_outstanding.csv')

In [59]:
## Connect to SQL database
conn = psycopg2.connect(database = 'cuzegotk',user = 'cuzegotk',password = 'NekW2BqJ8hW1wO3hCdpuEESPiP-y131V',host = 'raja.db.elephantsql.com')
cur = conn.cursor()
## Transfer CSV to SQL Database
for i in range(0,len(df)):
    cur.execute('INSERT INTO sharesoutstanding VALUES (%s, %s)', tuple(df.iloc[i,:].values))
    conn.commit()
cur.close()
conn.close()